In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### COAMPS data

### Parameters
coamps_dir = '/p/work1/lloveras/nov2018/coamps_files/'
nx_c = 301
ny_c = 201
nz_c = 45
lev1 = 500
lev2 = 600

### Latitude and longitude
lats = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats = np.reshape(lats, [ny_c, nx_c])
lons = np.reshape(lons, [ny_c, nx_c])
lons[lons > 0] -= 360

### Initial fields
pc_ctl = (np.fromfile(coamps_dir + 'ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pc_ctl = np.flip(np.reshape(pc_ctl, [nz_c, ny_c, nx_c]), axis=0)
tc_ctl = (np.fromfile(coamps_dir + 'pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
tc_ctl = np.flip(np.reshape(tc_ctl, [nz_c, ny_c, nx_c]), axis=0)
tl_ctl_1 = interplevel(tc_ctl, pc_ctl, lev1)
tl_ctl_2 = interplevel(tc_ctl, pc_ctl, lev2)
zc_ctl_1 = (np.fromfile(coamps_dir + 'geohgt_pre_000500_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=ny_c*nx_c))
zc_ctl_1 = np.reshape(zc_ctl_1, [ny_c, nx_c])
zc_ctl_2 = (np.fromfile(coamps_dir + 'geohgt_pre_000600_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=ny_c*nx_c))
zc_ctl_2 = np.reshape(zc_ctl_2, [ny_c, nx_c])

### First iteration
pc_1 = (np.fromfile(coamps_dir + '2x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pc_1 = np.flip(np.reshape(pc_1, [nz_c, ny_c, nx_c]), axis=0)
tc_1 = (np.fromfile(coamps_dir + '2x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
tc_1 = np.flip(np.reshape(tc_1, [nz_c, ny_c, nx_c]), axis=0)
tl_1_1 = interplevel(tc_1, pc_1, lev1)
tl_1_2 = interplevel(tc_1, pc_1, lev2)

### Fourth iteration
pc_4 = (np.fromfile(coamps_dir + '5x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pc_4 = np.flip(np.reshape(pc_4, [nz_c, ny_c, nx_c]), axis=0)
tc_4 = (np.fromfile(coamps_dir + '5x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
tc_4 = np.flip(np.reshape(tc_4, [nz_c, ny_c, nx_c]), axis=0)
tl_4_1 = interplevel(tc_4, pc_4, lev1)
tl_4_2 = interplevel(tc_4, pc_4, lev2)

### Eighth iteration
pc_8 = (np.fromfile(coamps_dir + '9x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
pc_8 = np.flip(np.reshape(pc_8, [nz_c, ny_c, nx_c]), axis=0)
tc_8 = (np.fromfile(coamps_dir + '9x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4',count=nz_c*ny_c*nx_c))
tc_8 = np.flip(np.reshape(tc_8, [nz_c, ny_c, nx_c]), axis=0)
tl_8_1 = interplevel(tc_8, pc_8, lev1)
tl_8_2 = interplevel(tc_8, pc_8, lev2)

In [ ]:
### Contour interval
zlvls1 = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/15
zlvls2 = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/10
zlvls3 = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/5

### Plot subdomain
min_lat = 17
max_lat = 62
min_lon = -130
max_lon = -70

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-101,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],['bottomleft','bottommiddle','bottomright'],['cbar1','cbar2','cbar3']],
                              constrained_layout=True, figsize=(8,5), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels = False

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zc_ctl_1,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topleft'].contourf(lons, lats, tl_1_1 - tl_ctl_1, levels=zlvls1,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('First Iteration')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
#############

gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zc_ctl_1,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im2 = axd['topmiddle'].contourf(lons, lats, tl_4_1 - tl_ctl_1, levels=zlvls2,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('Fourth Iteration')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topright'].contour(lons, lats, smooth2d(zc_ctl_1,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im3 = axd['topright'].contourf(lons, lats, tl_8_1 - tl_ctl_1, levels=zlvls3,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('Eighth Iteration')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zc_ctl_2,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im4 = axd['bottomleft'].contourf(lons, lats, tl_1_2 - tl_ctl_2, levels=zlvls1,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
#############

gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zc_ctl_2,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im5 = axd['bottommiddle'].contourf(lons, lats, tl_4_2 - tl_ctl_2, levels=zlvls2,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
#############

gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zc_ctl_2,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im6 = axd['bottomright'].contourf(lons, lats, tl_8_2 - tl_ctl_2, levels=zlvls3,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].text(0.0, 1.0, '(e)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar1 = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar1'])
cbar2 = fig.colorbar(im2, orientation='horizontal', cax=axd['cbar2'])
cbar3 = fig.colorbar(im3, orientation='horizontal', cax=axd['cbar3'])

plt.savefig('/p/work1/lloveras/nov2018/figs_png/diff_iters_6panels.png',bbox_inches='tight')
plt.show()
